In [1]:

from __future__ import annotations
from datetime import datetime, timezone
from syft_event import SyftEvents

from syft_core import Client
from syft_rds.models.dataset import Dataset, DatasetCreate
from syft_rds.service.dataset_service import DatasetService
from syft_rds.connection.connection import get_connection
from syft_rds.service.context import BaseRPCContext

In [2]:
client = Client.load()

In [3]:

from syft_core import SyftBoxURL


box = SyftEvents("my-rds-app")

# TODO: pass this in the handler
context = BaseRPCContext(client=client, box=box)  


@box.on_request("/apis/list")
def get_apis() -> list[str]:
    """Respond to a ping request."""
    return {SyftBoxURL.from_path(k, client.workspace): v for k, v in box._SyftEvents__rpc.items()}

@box.on_request("/datasets/create")
def create_dataset(dataset: DatasetCreate) -> Dataset:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.create_item(dataset)
    return res
    
@box.on_request("/datasets/list")
def list_datasets() -> list[Dataset]:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.list_items()
    return res


2025-02-07 19:16:05.123 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /apis/list
2025-02-07 19:16:05.125 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/create
2025-02-07 19:16:05.126 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/list


In [5]:
app_data_dir = client.my_datasite / "apps" / context.box.app_name
import shutil
shutil.rmtree(app_data_dir)

In [7]:
conn = get_connection(box, mock=True)   

In [8]:
# DatasetCreate(name="my-dataset", description="my dataset").to_item()

In [9]:
from typing import Any
from pydantic import BaseModel
from syft_rds.connection.connection import RPCConnection

class APIEndpoint(BaseModel):
    endpoint_path: str
    prefix: str
    api_callback: Any
    
    def __call__(self, body: dict | None = None, expiry: str = "5m", cache: bool = True):
        api = self.api_callback()
        res = api.conn.send(
            url=self.endpoint_path,
            body=body,
            expiry=expiry,
            cache=cache,
        )
        return res
    
    def __repr__(self):
        return f"APIEndpoint(prefix={self.prefix}, endpoint_path={self.endpoint_path})"
    def __repr_str__(self):
        return f"APIEndpoint(prefix={self.prefix}, endpoint_path={self.endpoint_path})"
    
def combine_path(prefix: str, path: str) -> str:
    if prefix and path:
        return f"{prefix}/{path}"
    elif prefix:
        return prefix
    else:
        return path

class APIModule(BaseModel):
    relative_module_path: str
    prefix: str
    submodules: dict[str, APIModule]
    endpoints: dict[str, APIEndpoint]
    api_callback: Any
    
    @classmethod
    def from_module_list(cls, prefix: str, relative_module_path: str, module_list: list[str], api_callback: Any) -> APIModule:
        submodules = {}
        endpoints = {}
        
        current_full_path = combine_path(prefix, relative_module_path)
        child_paths = [p for p in module_list if p.startswith(current_full_path) and p != current_full_path]
        for child_path in child_paths:
            child_path_rel_to_current = child_path[len(current_full_path)+1:] # +1 to remove the leading /            
            if "/" in child_path_rel_to_current:
                submodule_name = child_path_rel_to_current.split("/")[0]
                child_relative_module_path = combine_path(relative_module_path, submodule_name)
                submodules[submodule_name] = APIModule.from_module_list(prefix, child_relative_module_path, child_paths, api_callback=api_callback)
            else:
                endpoints[child_path_rel_to_current] = APIEndpoint(endpoint_path=child_path, prefix=prefix, api_callback=api_callback)
        return APIModule(prefix=prefix, relative_module_path=relative_module_path, submodules=submodules, endpoints=endpoints, api_callback=api_callback)

    
    def __getattr__(self, name: str) -> APIModule | APIEndpoint:
        if name in self.submodules:
            return self.submodules[name]
        elif name in self.endpoints:
            return self.endpoints[name]
        else:
            raise AttributeError(f"Module {self.relative_module_path} has no attribute {name}")

class API(BaseModel):
    email: str
    entry_module: APIModule | None = None
    conn: RPCConnection

    def from_email(email: str, conn: RPCConnection) -> API:
        prefix = f"syft://{email}/api_data/my-rds-app/rpc"
        res = conn.send(
            url=f"{prefix}/apis/list",
            body={},
            expiry="5m",
            cache=True,
        )
        paths = list([str(x) for x in res.keys()])
        _self = API(email=email, entry_module=None, conn=conn)
        def get_api():
            return _self
        
        entry_module = APIModule.from_module_list(prefix, "", paths, api_callback=get_api)
        _self.entry_module = entry_module
        return _self
    
    def __getattr__(self, name: str) -> APIModule | APIEndpoint:
        return getattr(self.entry_module, name)



In [10]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/apis/list",
    body={},
    expiry="5m",
    cache=True,
)

In [11]:
api = API.from_email(client.email, conn)

In [12]:
api.datasets.create(body=DatasetCreate(name="my-dataset", description="my dataset"))

full_json [
  {
    "uid": "a0c26121-abbc-4199-a401-57e974fb76df",
    "name": "my-dataset",
    "description": "my dataset"
  }
]


Dataset(uid=UUID('a0c26121-abbc-4199-a401-57e974fb76df'), name='my-dataset', description='my dataset')

In [17]:
api.datasets.list()

[Dataset(uid=UUID('a0c26121-abbc-4199-a401-57e974fb76df'), name='my-dataset', description='my dataset'),
 Dataset(uid=UUID('80774959-e620-42f9-a3c6-5bb020644877'), name='my-dataset', description='my dataset')]

In [14]:
res

{'syft://koen@openmined.org/api_data/my-rds-app/rpc/apis/list': <function __main__.get_apis() -> 'list[str]'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/create': <function __main__.create_dataset(dataset: 'DatasetCreate') -> 'Dataset'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/list': <function __main__.list_datasets() -> 'list[Dataset]'>}

In [15]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/datasets/create",
    body=DatasetCreate(name="my-dataset", description="my dataset"),
    expiry="5m",
    cache=True,
)

full_json [
  {
    "uid": "a0c26121-abbc-4199-a401-57e974fb76df",
    "name": "my-dataset",
    "description": "my dataset"
  },
  {
    "uid": "80774959-e620-42f9-a3c6-5bb020644877",
    "name": "my-dataset",
    "description": "my dataset"
  }
]


In [16]:
res

Dataset(uid=UUID('80774959-e620-42f9-a3c6-5bb020644877'), name='my-dataset', description='my dataset')